In [3]:
import os
import sys
#import folium
import pickle
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
#import osmnx as ox
#import networkx as nx
import matplotlib.pyplot as plt
import ipywidgets as widgets
import haversine as hs
from haversine import Unit
from IPython.display import display
from math import sin, cos, sqrt, atan2
from matplotlib.pyplot import figure
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, LabelEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, Lasso
from sklearn.ensemble import RandomForestRegressor

style = {'description_width': 'initial'}
pd.options.display.max_columns=100

In [2]:
df = pd.read_csv("finalTrain.csv")

In [3]:
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [4]:
df.isna().sum()

ID                                0
Delivery_person_ID                0
Delivery_person_Age            1854
Delivery_person_Ratings        1908
Restaurant_latitude               0
Restaurant_longitude              0
Delivery_location_latitude        0
Delivery_location_longitude       0
Order_Date                        0
Time_Orderd                    1731
Time_Order_picked                 0
Weather_conditions              616
Road_traffic_density            601
Vehicle_condition                 0
Type_of_order                     0
Type_of_vehicle                   0
multiple_deliveries             993
Festival                        228
City                           1200
Time_taken (min)                  0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45584 entries, 0 to 45583
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45584 non-null  object 
 1   Delivery_person_ID           45584 non-null  object 
 2   Delivery_person_Age          43730 non-null  float64
 3   Delivery_person_Ratings      43676 non-null  float64
 4   Restaurant_latitude          45584 non-null  float64
 5   Restaurant_longitude         45584 non-null  float64
 6   Delivery_location_latitude   45584 non-null  float64
 7   Delivery_location_longitude  45584 non-null  float64
 8   Order_Date                   45584 non-null  object 
 9   Time_Orderd                  43853 non-null  object 
 10  Time_Order_picked            45584 non-null  object 
 11  Weather_conditions           44968 non-null  object 
 12  Road_traffic_density         44983 non-null  object 
 13  Vehicle_conditio

In [6]:
df.describe()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Vehicle_condition,multiple_deliveries,Time_taken (min)
count,43730.000000,43676.000000,45584.000000,45584.000000,45584.000000,45584.000000,45584.000000,44591.000000,45584.000000
mean,29.566911,4.633774,17.017948,70.229684,17.465480,70.844161,1.023385,0.744635,26.293963
std,5.815064,0.334744,8.185674,22.885575,7.335562,21.120578,0.839055,0.572510,9.384298
min,15.000000,1.000000,-30.905562,-88.366217,0.010000,0.010000,0.000000,0.000000,10.000000
25%,25.000000,4.500000,12.933284,73.170000,12.988453,73.280000,0.000000,0.000000,19.000000
50%,30.000000,4.700000,18.551440,75.897963,18.633934,76.002574,1.000000,1.000000,26.000000
75%,35.000000,4.900000,22.728163,78.044095,22.785049,78.107044,2.000000,1.000000,32.000000
max,50.000000,6.000000,30.914057,88.433452,31.054057,88.563452,3.000000,3.000000,54.000000


In [7]:
df.duplicated().sum()

0

In [8]:
df.shape

(45584, 20)

### `Displacement`

In [9]:
def create_map(i):
    map_1 = folium.Map(location=[df.loc[i][4], df.loc[i][5]])
    folium.Marker([df.loc[i][4], df.loc[i][5]], popup='<i>Restaurant</i>', tooltip='Restaurant').add_to(map_1)
    folium.Marker([df.loc[i][6], df.loc[i][7]], popup='<i>Delevery Location</i>', tooltip='Delevery Location', icon=folium.Icon(color='red')).add_to(map_1)
    folium.PolyLine(locations=[[df.loc[i][4], df.loc[i][5]], [df.loc[i][6], df.loc[i][7]]],color='blue').add_to(map_1)
    return map_1

In [10]:
create_map(2)

In [38]:
loc1=(30.327968,78.046106)
loc2=(30.397968,78.116106)
print(hs.haversine(loc1,loc2, unit=Unit.KILOMETERS))

NameError: name 'hs' is not defined

In [12]:
list_1 = []
def displacement(i):
    loc1 = (df.loc[i][4], df.loc[i][5])
    loc2 = (df.loc[i][6], df.loc[i][7])
    list_1.append(hs.haversine(loc1,loc2, unit=Unit.KILOMETERS))

In [13]:
df.shape

(45584, 20)

In [14]:
for i in range(0, 45584):
    displacement(i)

NameError: name 'hs' is not defined

In [15]:
list_1[0:5]

[]

In [16]:
df['Displacement'] = pd.DataFrame(list_1)

ValueError: Cannot set a DataFrame with multiple columns to the single column Displacement

In [17]:
df['Displacement']

KeyError: 'Displacement'

In [18]:
df

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45579,0x7c09,JAPRES04DEL01,30.0,4.8,26.902328,75.794257,26.912328,75.804257,24-03-2022,11:35,11:45,Windy,High,1,Meal,motorcycle,0.0,No,Metropolitian,32
45580,0xd641,AGRRES16DEL01,21.0,4.6,0.000000,0.000000,0.070000,0.070000,16-02-2022,19:55,20:10,Windy,Jam,0,Buffet,motorcycle,1.0,No,Metropolitian,36
45581,0x4f8d,CHENRES08DEL03,30.0,4.9,13.022394,80.242439,13.052394,80.272439,11-03-2022,23:50,24:05:00,Cloudy,Low,1,Drinks,scooter,0.0,No,Metropolitian,16
45582,0x5eee,COIMBRES11DEL01,20.0,4.7,11.001753,76.986241,11.041753,77.026241,07-03-2022,13:35,13:40,Cloudy,High,0,Snack,motorcycle,1.0,No,Metropolitian,26


### `Time`

In [19]:
for i in df["Time_Order_picked"]:
    if i < str(0.999999999):
        print(i)

0.458333333
0.75
0.958333333
0.916666667
0.875
0.875
0.958333333
0.375
0.833333333
0.708333333
0.75
0.625
0.75
0.791666667
0.375
0.708333333
0.958333333
0.458333333
0.958333333
0.791666667
0.75
0.5
0.833333333
0.5
0.916666667
0.75
0.958333333
0.541666667
0.875
0.75
0.916666667
0.5
0.5
0.833333333
0.916666667
0.416666667
0.791666667
0.75
0.833333333
0.833333333
0.875
0.875
0.416666667
0.5
0.375
0.833333333
0.958333333
0.875
0.916666667
0.458333333
0.75
0.958333333
0.375
0.875
0.916666667
0.875
0.541666667
0.958333333
0.833333333
0.75
0.833333333
0.875
0.916666667
0.958333333
0.875
0.625
0.708333333
0.458333333
0.458333333
0.75
0.791666667
0.5
0.916666667
0.958333333
0.875
0.916666667
0.75
0.958333333
0.375
0.916666667
0.875
0.916666667
0.875
0.75
0.791666667
0.833333333
0.791666667
0.791666667
0.833333333
0.875
0.708333333
0.5
0.791666667
0.916666667
0.833333333
0.458333333
0.916666667
0.875
0.75
0.916666667
0.5
0.75
0.791666667
0.875
0.958333333
0.75
0.916666667
0.833333333
0.5
0.875
0

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45584 entries, 0 to 45583
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45584 non-null  object 
 1   Delivery_person_ID           45584 non-null  object 
 2   Delivery_person_Age          43730 non-null  float64
 3   Delivery_person_Ratings      43676 non-null  float64
 4   Restaurant_latitude          45584 non-null  float64
 5   Restaurant_longitude         45584 non-null  float64
 6   Delivery_location_latitude   45584 non-null  float64
 7   Delivery_location_longitude  45584 non-null  float64
 8   Order_Date                   45584 non-null  object 
 9   Time_Orderd                  43853 non-null  object 
 10  Time_Order_picked            45584 non-null  object 
 11  Weather_conditions           44968 non-null  object 
 12  Road_traffic_density         44983 non-null  object 
 13  Vehicle_conditio

### `Dealing With null values`

In [21]:
df.isna().sum()

ID                                0
Delivery_person_ID                0
Delivery_person_Age            1854
Delivery_person_Ratings        1908
Restaurant_latitude               0
Restaurant_longitude              0
Delivery_location_latitude        0
Delivery_location_longitude       0
Order_Date                        0
Time_Orderd                    1731
Time_Order_picked                 0
Weather_conditions              616
Road_traffic_density            601
Vehicle_condition                 0
Type_of_order                     0
Type_of_vehicle                   0
multiple_deliveries             993
Festival                        228
City                           1200
Time_taken (min)                  0
dtype: int64

In [22]:
df['Delivery_person_Age'].mean()

29.566910587697233

In [23]:
df['Delivery_person_Age'].fillna(df['Delivery_person_Age'].mean(), inplace=True)

In [24]:
df.Delivery_person_Ratings.mean()

4.6337736972250205

In [25]:
df['Delivery_person_Ratings'].fillna(df['Delivery_person_Ratings'].mean(), inplace=True)

In [26]:
df.loc[df['City'].isna(), 'City'] = 'Semi-Urban'

In [27]:
df.loc[df['multiple_deliveries'].isna(), 'multiple_deliveries'] = 3.0

In [28]:
df.loc[df['Weather_conditions'].isna()]

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
65,0xa9f,BANGRES13DEL01,29.566911,4.633774,12.935662,77.614130,12.975662,77.654130,11-03-2022,NaN,15:30,NaN,NaN,1,Drinks,scooter,0.0,No,Metropolitian,19
111,0x6e2,VADRES06DEL01,29.566911,4.633774,22.312790,73.170283,22.422790,73.280283,02-04-2022,NaN,18:25,NaN,NaN,3,Snack,electric_scooter,0.0,No,Metropolitian,25
241,0x1b50,PUNERES16DEL01,29.566911,4.633774,18.536718,73.830327,18.646718,73.940327,08-03-2022,NaN,21:25,NaN,NaN,3,Buffet,motorcycle,1.0,Yes,Urban,43
339,0x4c1,BANGRES13DEL02,29.566911,4.633774,12.935662,77.614130,13.065662,77.744130,02-03-2022,NaN,24:05:00,NaN,NaN,3,Drinks,scooter,1.0,No,Metropolitian,26
416,0xc003,DEHRES13DEL02,29.566911,4.633774,-30.366322,-78.070453,30.496322,78.200453,18-02-2022,NaN,22:30,NaN,NaN,3,Snack,scooter,1.0,No,Metropolitian,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45323,0x2950,VADRES04DEL02,50.000000,6.000000,0.000000,0.000000,0.020000,0.020000,19-03-2022,NaN,10:40,NaN,NaN,3,Drinks,electric_scooter,1.0,No,Urban,28
45385,0xdc4,RANCHIRES02DEL03,29.566911,4.633774,0.000000,0.000000,0.030000,0.030000,05-03-2022,NaN,17:25,NaN,NaN,3,Buffet,bicycle,1.0,No,Metropolitian,21
45495,0x461,BANGRES07DEL02,29.566911,4.633774,-12.978453,-77.643685,12.998453,77.663685,17-03-2022,NaN,09:30,NaN,NaN,3,Meal,scooter,1.0,No,Semi-Urban,24
45509,0x863,INDORES05DEL01,29.566911,4.633774,22.727021,75.884167,22.737021,75.894167,19-03-2022,NaN,11:40,NaN,NaN,3,Snack,electric_scooter,0.0,No,Metropolitian,25


In [29]:
df.loc[df['Festival'].isna(), 'Festival'] = 'No'

In [30]:
df.loc[df['Weather_conditions'].isna(), 'Weather_conditions'] = 'Sunny'

In [31]:
df.loc[df['Road_traffic_density'].isna(), 'Road_traffic_density'] = 'Low'

In [32]:
df['City'].unique()

array(['Metropolitian', 'Urban', 'Semi-Urban'], dtype=object)

## `EDA`

In [33]:
def value_count_norm(df, feature):
    value_count = df[feature].value_counts()
    value_count_norm_1 = df[feature].value_counts(normalize=True) * 100
    value_count_norm_1_concat = pd.concat([value_count, value_count_norm_1], axis=1)
    value_count_norm_1_concat.columns = ['Count', 'Frequency %']
    return value_count_norm_1_concat

In [34]:
def create_pie_chart(df, feature):
    n = len(value_count_norm(df, feature).index)
    colors = sns.color_palette('pastel')[0:n]
    plt.pie(value_count_norm(df, feature).iloc[:, 1],
            colors = colors, autopct='%1.2f%%', 
            labels=value_count_norm(df, feature).index)
    plt.title('{}'.format(feature))
    return plt.show()

In [35]:
def create_hist_plot(df, feature):
    figure(figsize=(20, 8))
    sns.histplot(df[feature], kde=True)
    plt.title('{}'.format(feature))
    return plt.show()

In [36]:
def get_info(df, feature):
    match feature:
        case 'Weather_conditions'| 'Road_traffic_density'| 'Vehicle_condition'| 'Type_of_order'| 'Type_of_vehicle'| 'multiple_deliveries'| 'Festival'| 'City':
            print('Description:\n{}'.format(df[feature].describe()))
            print('*'*50)
            print('Object type:\n{}'.format(df[feature].dtype))
            print('*'*50)
            print('Value count:\n{}'.format(value_count_norm(df, feature)))
        case 'Delivery_person_Age'| 'Delivery_person_Ratings':
            print('Description:\n{}'.format((np.abs(df[feature])).describe()))
            print('*'*50)
            print('Object type:{}'.format(df[feature].dtype))
        case _:
            print('Error')

SyntaxError: invalid syntax (2303448048.py, line 2)

In [37]:
get_report(df, 'Weather_conditions')

NameError: name 'get_report' is not defined

In [ ]:
get_report(df, 'Road_traffic_density')

In [ ]:
get_report(df, 'Vehicle_condition')

In [ ]:
get_report(df, 'Type_of_order')

In [ ]:
get_report(df, 'Type_of_vehicle')

In [ ]:
get_report(df, 'multiple_deliveries')

In [ ]:
get_report(df, 'Festival')

In [ ]:
get_report(df, 'City')

In [ ]:
get_report(df, 'Delivery_person_Age')

In [ ]:
get_report(df, 'Delivery_person_Ratings')

In [ ]:
def drop_features(df, feature):
    df = df.drop([feature], axis=1, inplace=True)
    return df

In [ ]:
drop_features(df, 'ID')
drop_features(df, 'Delivery_person_ID')
drop_features(df, 'Restaurant_latitude')
drop_features(df, 'Restaurant_longitude')
drop_features(df, 'Delivery_location_latitude')
drop_features(df, 'Delivery_location_longitude')
drop_features(df, 'Order_Date')
drop_features(df, 'Time_Orderd')
drop_features(df, 'Time_Order_picked')

In [ ]:
df

In [ ]:
df['City'].unique()

In [ ]:
figure(figsize=(20, 12))
mask = np.zeros_like(df.corr('spearman'))
mask[np.triu_indices_from(mask)] = True
sns.heatmap(df.corr('spearman'), annot=True, cmap='flare', linewidths=.5, mask=mask)

In [ ]:
df.head()

In [ ]:
class encoding(BaseEstimator, TransformerMixin):
    def __init__(self, columns=['Weather_conditions', 'Road_traffic_density', 'Type_of_order', 'Type_of_vehicle', 'Festival', 'City']):
        self.columns = columns
    def fit(self, df):
        return self
    def transform(self, df):
        oe = OrdinalEncoder()
        df[self.columns] = oe.fit_transform(df[self.columns])
        return df

In [ ]:
class feature_scaling(BaseEstimator, TransformerMixin):
    def __init__(self, columns=['Delivery_person_Age', 'Delivery_person_Ratings', 'Displacement']):
        self.columns = columns
    def fit(self, df):
        return self
    def transform(self, df):
        sc = StandardScaler()
        df[self.columns] = sc.fit_transform(df[self.columns])
        return df

In [ ]:
df.info()

class data_conv(BaseEstimator, TransformerMixin):
    def __

In [ ]:
pipe = Pipeline([
    ('encoding', encoding()),
    ('feature_scaling', feature_scaling())
])

In [ ]:
pipe

In [ ]:
pipe.fit_transform(df)

In [ ]:
df['Vehicle_condition'] = df['Vehicle_condition'].astype(float)

In [ ]:
df.columns

In [ ]:
x = df.drop(['Time_taken (min)'], axis=1)
y = df['Time_taken (min)']

In [ ]:
x

In [ ]:
x.boxplot()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
models = {
    "LinearRegression": LinearRegression(n_jobs=-1),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "RandomForest": RandomForestRegressor()
}

In [ ]:
def train_model(model, model_name, x_train= x_train, x_test = x_test, y_train = y_train, y_test = y_test):
    print("#"*50)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(model_name,":")
    print("r2_score", r2_score(y_test, y_pred))
    print("Score", ":", mean_squared_error(y_pred, y_test))
    print("Mean score", ":", np.sqrt(mean_squared_error(y_pred, y_test)))

In [ ]:
for model_name, model in models.items():
    train_model(model, model_name)